In [2]:
!pip install datasets evaluate google-auth
!pip install rouge_score
import requests
import vertexai
from vertexai.generative_models import GenerativeModel
import json
from google.colab import auth
from datasets import load_dataset
import evaluate
auth.authenticate_user()
vertexai.init(project="medicaldataextraction", location="us-central1")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=r

In [4]:
model = vertexai.generative_models.GenerativeModel("gemini-1.5-flash")
print("Gemini 1.5 Flash model initialized successfully!")

Gemini 1.5 Flash model initialized successfully!


In [3]:
def fetch_dataset():
    dataset = load_dataset("AGBonnet/augmented-clinical-notes", split="train")

    if len(dataset) > 0:
        first_example = dataset[0]
        conversation = first_example['conversation']
        summary = first_example['summary']

        print("Conversation:\n", conversation, "\n")
        print("Summary:\n", summary, "\n")
        return conversation, summary
    else:
        print("Dataset is empty.")
        return None, None

conversation, summary = fetch_dataset()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.76k [00:00<?, ?B/s]

augmented_notes_30K.jsonl:   0%|          | 0.00/372M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Conversation:
 Doctor: Good morning, what brings you to the Outpatient department today?
Patient: Good morning doctor, I have some discomfort in my neck and lower back, and I'm not able to maintain an erect posture.
Doctor: Hmm, okay. Can you tell me more about the discomfort?
Patient: Yes, I tend to fall on either side when I stand up from a sitting position, and my head is always turned to the right and upwards.
Doctor: I see. Are you experiencing any pain in your neck?
Patient: Yes, I have pain and discomfort in my neck.
Doctor: Okay. And what about your back?
Patient: There is a sideways bending in my lumbar region. To counter the abnormal positioning of my back and neck, I have to keep my limbs in a specific position to allow my body weight to be supported.
Doctor: I understand. Does this restriction of body movements affect your daily life?
Patient: Yes, I need assistance in standing and walking, and my parents have to help me with my daily chores, including all activities of sel

In [ ]:
def fetch_response(prompt):
    system_prompt = f"""You are an advanced AI trained to extract vital medical information from unstructured text.
    Given the conversation text below, please extract the relevant details to fill in the specified schema accurately.
    {prompt}
    """
    return model.generate_content(system_prompt).text

def fetch_response1(prompt):
    system_prompt = f"""Analyze the conversation text below and extract the relevant details to accurately fill in the specified schema.
    {prompt}
    """
    return model.generate_content(system_prompt).text

print("Response functions defined!")


Response functions defined!


In [ ]:
def extract_information(conversation):
    schema = {
        "visit motivation": "None",
        "admission": [
            {
                "reason": "None",
                "date": "None",
                "duration": "None",
                "care center details": "None"
            }
        ],
        "patient information": {
            "age": "None",
            "sex": "None",
            "ethnicity": "None",
            "weight": "None",
            "height": "None",
            "family medical history": "None",
            "recent travels": "None",
            "socio economic context": "None",
            "occupation": "None"
        },
        "patient medical history": {
            "physiological context": "None",
            "psychological context": "None",
            "vaccination history": "None",
            "allergies": "None",
            "exercise frequency": "None",
            "nutrition": "None",
            "sexual history": "None",
            "alcohol consumption": "None",
            "drug usage": "None",
            "smoking status": "None"
        },
        "surgeries": [
            {
                "reason": "None",
                "Type": "None",
                "time": "None",
                "outcome": "None",
                "details": "None"
            }
        ],
        "symptoms": [
            {
                "name of symptom": "None",
                "intensity of symptom": "None",
                "location": "None",
                "time": "None",
                "temporalisation": "None",
                "behaviours_affecting_the_symptom": "None",
                "details": "None"
            }
        ],
        "medical examinations": [
            {
                "name": "None",
                "result": "None",
                "details": "None"
            }
        ],
        "diagnosis_tests": [
            {
                "test": "None",
                "severity": "None",
                "result": "None",
                "condition": "None",
                "time": "None",
                "details": "None"
            }
        ],
        "treatments": [
            {
                "name": "None",
                "related condition": "None",
                "dosage": "None",
                "time": "None",
                "frequency": "None",
                "duration": "None",
                "reason for taking": "None",
                "reaction to treatment": "None",
                "details": "None"
            }
        ],
        "discharge": {
            "reason": "None",
            "referral": "None",
            "follow up": "None",
            "discharge summary": "None"
        }
    }
    prompt_summary = f"""
    Extract the following details from the conversation text: "{conversation}"
    Fill in the schema as follows:
    {json.dumps(schema, indent=4)}
    """
    filled_schema = fetch_response(prompt_summary)
    cleaned_schema = filled_schema.replace("```json", "").replace("```", "").strip()
    cleaned_schema = cleaned_schema.replace("'", '"')

    try:
        json_schema = json.loads(cleaned_schema)
        print("### Extracted Information:\n")
        print(json.dumps(json_schema, indent=4))
        return json_schema
    except json.JSONDecodeError as e:
        print(f"Error parsing response as JSON: {e}")
        print("Raw response:\n", cleaned_schema)
        return None

extracted_info = extract_information(conversation)


### Extracted Information:

{
    "visit motivation": "Discomfort in neck and lower back, inability to maintain an erect posture",
    "admission": [
        {
            "reason": "None",
            "date": "None",
            "duration": "None",
            "care center details": "Outpatient department"
        }
    ],
    "patient information": {
        "age": "None",
        "sex": "None",
        "ethnicity": "None",
        "weight": "None",
        "height": "None",
        "family medical history": "None",
        "recent travels": "None",
        "socio economic context": "None",
        "occupation": "None"
    },
    "patient medical history": {
        "physiological context": "None",
        "psychological context": "Bipolar affective disorder diagnosed 7 years ago",
        "vaccination history": "None",
        "allergies": "None",
        "exercise frequency": "None",
        "nutrition": "None",
        "sexual history": "None",
        "alcohol consumption": "None

In [ ]:
def compute_rouge(generated_summary, reference_summary):
    rouge = evaluate.load('rouge')
    results = rouge.compute(predictions=[generated_summary], references=[reference_summary])

    print("### ROUGE Scores:\n")
    for key, value in results.items():
        print(f"{key}: {value:.4f}")

if extracted_info and summary:
    generated_summary = json.dumps(extracted_info, indent=4)
    compute_rouge(generated_summary, summary)
else:
    print("Unable to compute ROUGE score. Missing data.")

### ROUGE Scores:

rouge1: 0.7673
rouge2: 0.5675
rougeL: 0.6259
rougeLsum: 0.7456


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def extract_info_from_input(conversation):
    extracted_info = extract_information(conversation)

    if extracted_info is not None:
        return json.dumps(extracted_info, indent=4)
    else:
        return "Error extracting information."

interface = gr.Interface(
    fn=extract_info_from_input,
    inputs="text",
    outputs="json",
    title="Medical Information Extraction",
    description="Enter a conversation to extract medical information based on a predefined schema."
)

interface.launch(share=True)


In [6]:
prompt = f"Extract medical details from this conversation: {conversation}"
response = model.generate_content(prompt).text
print(response)

## Extracted Medical Details:

**Patient's Presenting Symptoms:**

* Neck pain and discomfort
* Lower back pain and discomfort
* Difficulty maintaining erect posture
* Falls to either side when standing up from sitting position
* Head turned to the right and upwards
* Sideways bending in the lumbar region
* Requires assistance with standing, walking, and daily chores

**Patient's Medical History:**

* Diagnosed with bipolar affective disorder 7 years ago
* First episode of mania at age 11
* Taking olanzapine tablets for 7 years (2.5-10 mg per day)
* Developed neck pain and discomfort within 2 weeks of starting olanzapine (5 mg per day)
* Neck muscle contractions causing head to pull right and upwards
* Symptoms persisted for 3 years with varying intensity, distress, and dysfunction correlating with olanzapine dosage
* Briefly prescribed trihexyphenidyl 4 mg per day for rigidity in upper limbs, showing good response

**Possible Diagnoses:**

* **Dystonia:**  The patient's symptoms, part